Run the following command:

pip install -r requirements.txt

In [194]:
import re
import tweepy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
#from tensorflow.python.keras.utils import _____
from keras.layers import Embedding,LSTM,Dense,Dropout
from sklearn.model_selection import train_test_split

## Importing all the relevant libraries

In [195]:
from dotenv import load_dotenv, dotenv_values
secrets = dotenv_values(".env")

#Imports the credentials for Twitter

consumerKey = secrets["consumerKey"]
consumerSecret = secrets["consumerSecret"]
accessToken = secrets["accessToken"]
accessSecret = secrets["accessSecret"]

In [196]:
authenticate = tweepy.OAuthHandler(consumerKey,consumerSecret)
authenticate.set_access_token(accessToken,accessSecret)

api = tweepy.API(authenticate,wait_on_rate_limit= True)

In [197]:
mention = api.mentions_timeline()
print(mention)

for tweets in mention:
  print(str(tweets.id)+'-'+tweets.text)

[]


In [198]:
dftweet = pd.DataFrame(['I hate him'],columns = ["Column2"])
dftweet.head()

,Column2
0,I hate him


In [199]:
#clean tweets
def cleanTxt(text):
  text =re.sub(r'@[A-Za-z0-9]+','',text) #remove @
  text = re.sub(r'#','',text)# remove #
  text = re.sub(r'RT[\s]+','',text)# remove RT
  text = re.sub(r'https?:\/\/\S+','',text)#remove hyperlink

  return text

dftweet['Column2']= dftweet['Column2'].apply(cleanTxt)
print(dftweet)

      Column2
0  I hate him


In [200]:
df = pd.read_csv('combined.csv')
df = pd.read_csv('new.csv')
df.Column1 =df.Column1.astype(str)
df.Column2 =df.Column2.astype(str)
df['Column1']= df['Column1'].apply(cleanTxt)
df.head()
#df.shape

,Column1,Column2
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [201]:
indexNames = df[ df['Column2'] == 'nan' ].index
df.drop(indexNames , inplace=True)
df['Column2'].value_counts()

joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: Column2, dtype: int64

In [202]:

tokenizer = Tokenizer(num_words=5000,split =" ")
tokenizer.fit_on_texts(df['Column1'].values)

df_x = tokenizer.texts_to_sequences(df['Column1'].values)
df_x = pad_sequences(df_x)
print(df_x[:5])

tokenizer.fit_on_texts(dftweet['Column2'].values)
x_tweet = tokenizer.texts_to_sequences(dftweet['Column2'].values)
x_tweet = pad_sequences(x_tweet)

print(df_x.shape)

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    1  138    2  678]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     1   39  100   59    7   14  493    4   14 3495  552   31   59   60
   127  147   75 1479    3   21 1254]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0   16 3059    6
  1148    4  285    1    2  494  437]
 [   0    0    0    0 

In [203]:
df_y = pd.get_dummies(df['Column2']).values
[print(df['Column2'][i],df_y[i]) for i in range(101,135)]
print(df_x.shape)
print(df_y.shape)
x_train,x_test,y_train,y_test = train_test_split(df_x,df_y,test_size = 0.2,random_state = 0)

joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
sadness [0 0 0 0 1 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
sadness [0 0 0 0 1 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
love [0 0 0 1 0 0]
sadness [0 0 0 0 1 0]
sadness [0 0 0 0 1 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
sadness [0 0 0 0 1 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
fear [0 1 0 0 0 0]
fear [0 1 0 0 0 0]
fear [0 1 0 0 0 0]
sadness [0 0 0 0 1 0]
love [0 0 0 1 0 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
love [0 0 0 1 0 0]
fear [0 1 0 0 0 0]
surprise [0 0 0 0 0 1]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
joy [0 0 1 0 0 0]
anger [1 0 0 0 0 0]
(16000, 63)
(16000, 6)


In [204]:

model = Sequential()
model.add(Embedding(5000,256,input_length=df_x.shape[1]))
#model.add(Dense(4, activation='softmax'))
model.add(Dropout(0.3))
model.add(LSTM(256,return_sequences=True,dropout=0.3,recurrent_dropout=0.2))
model.add(LSTM(256,dropout=0.3,recurrent_dropout=0.2))
model.add(Dense(6,activation='softmax'))

In [193]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 63, 256)           1280000   
_________________________________________________________________
dropout_10 (Dropout)         (None, 63, 256)           0         
_________________________________________________________________
lstm_20 (LSTM)               (None, 63, 256)           525312    
_________________________________________________________________
lstm_21 (LSTM)               (None, 256)               525312    
_________________________________________________________________
dense_10 (Dense)             (None, 6)                 1542      
Total params: 2,332,166
Trainable params: 2,332,166
Non-trainable params: 0
_________________________________________________________________


In [149]:
model.fit(x_train, y_train,epochs=10,batch_size =32,verbose=2)

Epoch 1/10
400/400 - 421s - loss: 1.1831 - accuracy: 0.5562
Epoch 2/10
400/400 - 408s - loss: 0.3216 - accuracy: 0.8905
Epoch 3/10
400/400 - 396s - loss: 0.1500 - accuracy: 0.9432
Epoch 4/10
400/400 - 394s - loss: 0.1036 - accuracy: 0.9566
Epoch 5/10
400/400 - 399s - loss: 0.0884 - accuracy: 0.9630
Epoch 6/10
400/400 - 394s - loss: 0.0718 - accuracy: 0.9694
Epoch 7/10
400/400 - 392s - loss: 0.0632 - accuracy: 0.9766
Epoch 8/10
400/400 - 394s - loss: 0.0607 - accuracy: 0.9759
Epoch 9/10
400/400 - 392s - loss: 0.0510 - accuracy: 0.9792
Epoch 10/10
400/400 - 395s - loss: 0.0434 - accuracy: 0.9834


In [150]:
model.save('sentiment')

INFO:tensorflow:Assets written to: sentiment\assets


In [171]:
# LOOKS AT THE CONTENT OF THE MENTIONED TWEET
#tweets = api.mentions_timeline(tweet_mode='extended')
#a = tweets[3].in_reply_to_status_id_str
#tweet_a = api.get_status(id=a)
#print(tweet_a.text)
def check (i):
    print(i)
    print(np.max(i))
    a = np.where(i == np.max(i))
    a = a[0][0]
    if ( a == 0):
        return('Anger')
    elif (a == 1):
        return('Fear')
    elif (a ==2):
        return('Joy')
    elif (a ==3):
        return('Love')
    elif (a ==4):
        return('Sadness')   
    else:
        return('Surprised')

predictions = model.predict(x_tweet)
print (dftweet.Column2[0])
#print ( (np.where(predictions[0] == np.max(predictions[0])))[0][0])
#[print(df['Column1'][i],predictions[i],y_test[i]) for i in range (0,1)]
#print(check(y_test[0]))
print(check(predictions[0]))

I hate him
[0.14295103 0.14247619 0.14279318 0.14279813 0.14272663 0.14315179
 0.14310299]
0.14315179
Sadness
